* Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). Or, alternatively, **Restart & Run All**.

* Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

* You can always add additional cells to the notebook to experiment, to test your answers, or to provide additional support for your answers.

* You should not need to install new packages to complete an assignment. If you use any packages not available via the provided `Project.toml`, or if you change the `Manifest.toml`, then your assignment will likely not be graded correctly.

* Submissions are only accepted via CANVAS!

* Late submissions will not be accepted and receive 0%. There will be no exceptions.

* By entering your name below you confirm that you have completed this assignment on your own. Suspicion of plagiarism / copying may result in an ad hoc oral examination on the content of the assignment. I may then downgrade an assignment. 

In [ ]:
NAME = ""

---

# MATH 405/607 

# Numerical Methods for Differential Equations

## Assignment 2: Nonlinear systems, interpolation, quadrature


#### Notes

* This assignment will count for 10% of your total grade.

* We will start to be rigorous about following instructions precisely to enable the autograder to correctly read your answers. Please watch out for those instructions. If a solution is correct but is not stored in the correct variables we will only give partial points. 



In [ ]:
include("math405.jl")

### Question 1 [5+5]

Skim the `README` files of [`Roots.jl`](https://github.com/JuliaMath/Roots.jl) and of [NLsolve.jl](https://github.com/JuliaNLSolvers/NLsolve.jl) to understand what these packages are written for. Then use appropriate functions provided by these packages to solve the following nonlinear equations: 

(a) Find all solutions $x \in [0.1, 4.1\pi]$ of 
$$
    x^{-2} = \sin(x)
$$
Assign these to a vector as follows: `Xa = [ x1, x2, ... ]`

(b) Find the unique solution of the system  
$$\begin{aligned}
    & f(x) := \nabla \varphi(x) = 0,  \qquad \text{where} 
    & \varphi(x) = x_1^4 + \sum_{j = 2}^{11} (x_j - x_{j-1})^4 + x_{11}^4 + 0.01 \sum_{j = 1}^{11} x_j.
\end{aligned}$$
and store it in the variable `xb` i.e. 

HINT: you need not derive and implement the gradient, but could use an AD package (e.g. `ForwardDiff.jl`) to obtain it from $\varphi$, e.g., 
```julia 
phi(x) = ...
fb(x) = ForwardDiff.gradient(phi, x)
```

In [ ]:
# Somewhere in this code you should have the lines 
#  Xa = [ ... ]
#  xb =  ...

# YOUR CODE HERE

### Question 2 [10+5+10]

Newton's method is chaotic (cf Fractals) and in general converges only locally, or at least its global behaviour is unpredictable for most practical purposes. Most implementations of Newton's method therefore have some "globalisation" strategies, i.e. incorporate ideas (heuristic and rigorous) to improve global convergence properties, or at least increase make the behaviour more predictable. In this question we will explore one strategy of this kind. 

Suppose your current iterate is $x_n$ then the next iterate would be $x_{n+1} = x_n - \partial f(x_n)^{-1} f(x_n)$. But instead, let us define this increment as a *search vector*. That is, we define 
$$
    p_n := - \partial f(x_n)^{-1} f(x_n)
$$
and look for updates of the form 
$$
    x_{n+1} = x_n + \alpha_n p_n
$$
where $\alpha_n \in (0, 1]$.

(a) Compute $\frac{d}{d\alpha} |f(x_n + \alpha p_n)|^2$ at $\alpha = 0$, then deduce that, for sufficiently small $\alpha$ the update $x_{n+1}$ satisfies
$$
  \text{(DEC)} \qquad   |f(x_{n+1})| \leq (1 - \alpha_n/2) |f(x_n)|
$$
You may use any regularity on $f$ that you need.

Remarks:
* In your proof you should find that the factor $1/2$ in $(1-\alpha/2)$ is arbitrary. But it is a sensible choice that seems to work quite well in the following tests.
* You can proceed to part (b) without answering this question.

YOUR ANSWER HERE

(b) To achieve the (DEC) condition, we can use a backtracking algorithm. At each iterate $x_n$ our first guess should be $\alpha = 1$ to obtain quadratic convergence in the limit. If this fails the (DEC) condition, then we halve $\alpha$ until it is satisfied.
``` 
WHILE ||f(x + alpha p)|| > (1-alpha/2) * ||f(x)||
    alpha <- alpha / 2
```
* Implement this backtracking condition into a Newton iteration. 
* In light of part (a) of this question, terminate the backtracking with `return nothing` when $\alpha < 10^{-8}$.

In the code-cell below most of the two Newton methods have already been written. Only edit the part of the code that is indicated to incorporate the backtracking loop. Just translate the pseudocode into valid Julia code.

In [ ]:

function newton(f, x0, tol, df = x -> ForwardDiff.jacobian(f, x); maxiter = 10)
    x = x0 
    it = 0
    while norm(f(x)) > tol 
        x -= df(x) \ f(x)
        it += 1; 
        if (it > maxiter) || any(isnan, x) || any(isinf, x)
            return nothing
        end
    end 
    return x, it 
end

function damped_newton(f, x0, tol, df = x -> ForwardDiff.jacobian(f, x); maxiter = 100)
    x = x0
    it = 0

    while norm(f(x), Inf) > tol 
        p = - (df(x) \ f(x))
        
        # --------- Backtracking loop 
        # YOUR CODE HERE
        # ---------------------------
        
        x += α * p
        it += 1

        # for debugging!
        # @show α, norm(f(x), Inf)    

        if it > maxiter; return nothing; end 
    end
    return x, it 
end

In [ ]:
# this cell is not part of the assignment but is intended to help you 
# test that your code is correct. 
# you may edit this cell to experiment with your code ...

println("""
 This little test shows how Newton's method can converge 
 predictably or to vastly different solution
 while the damped Newton method usually converges predictably.
    
 Check that the damped Newton method always converges to the 
 root near 6.38.
""")

# wrapping a scalar problem into a vectorial one (with 1 dimension of course...)
fbes = x -> [besselj(3,x[1])]

println("Newton - Start guess 4.85:")
xn1, itn1 = newton(fbes, [4.85], 1e-10)
println("   x = $(xn1), #it = $(itn1)")

println("Newton - Start guess 4.84:")
xn2, itn2 = newton(fbes, [4.84], 1e-10)
println("   x = $(xn2), #it = $(itn2)")

println("Damped Newton - Start guess 4.85:")
xd1, itd1 = damped_newton(fbes, [4.85], 1e-10)
println("   x = $(xd1), #it = $(itd1)")

println("Damped Newton - Start guess 4.84:")
xd2, itd2 = damped_newton(fbes, [4.84], 1e-10)
println("   x = $(xd2), #it = $(itd2)")

plot(x->besselj(3,x), 0, 20, lw=3, label="", grid=:xy, size=(500,150), legend = :outertopright)
scatter!([4.84], fbes(4.84), label = "Starting guess")
scatter!(xn2, [0.0], label = "Newton")
scatter!(xd2, [0.0], label = "Damped Newton")

(c) Use both the Newton and damped Newton algorithms to solve the problem from Question 1, with starting guess `x0[i] = c * i * (12-i)` where `c in [-0.1, -0.01]` and briefly comment on your observations. Use comments such as ,
```julia 
# we oberserve that `newton` ... while `damped_newton` ... 
```

In [ ]:
# YOUR CODE HERE

### Question 3: Simpson Rule [5+5+5+10]

Consider the quadrature rule  (Simpson rule) 
$$ 
    \int_{-h/2}^{h/2} f(x) \,dx \approx \frac{h}{6} \big( f(-h/2) + 4 f(0) + f(h/2) \big)
$$

(a) We said that most quadrature rules can be interpreted as integrating a polynomial interpolant of the integrand. Which polynomial interpolant does the Simpson rule correspond to? (state without proof)

(b) Derive an error bound, assuming that $f \in C^4([a, b])$.  
[Full points for a short proof that gets within a factor 2 of the sharp bound; partial points for a proof that gets the correct order.]

(c) State (without proof) the corresponding estimate for the composite Simpson rule with mesh-size $h$.

YOUR ANSWER HERE

(d) Using the integral $\int_0^\pi x \sin(x) \,dx = \pi$ as an example, numerically confirm the convergence rate predicted in (b): Produce a figure that compares the numerical convergence rate against the predicted rate. 

Ideally, you should first implement a function, e.g., `function simpson(f, a, b, N)` which implements the simpson rule. Then implement a short test (see lectures for inspiration!) which checks that the output of your function converges with the predicted rate to the analytic value.

In [ ]:
# solution part (d) 

# YOUR CODE HERE

## Question 4 [5+5+5+5] : integrate some functions

Integrate each of the following functions numerically. For each function you should 
* use an integrator that you implemented yourself (e.g., simpson from question 3?)
* use the `Cubature.jl` package. 
Store the solutions in the variables `I_x` (your own method) and `Icub_x` (the `Cubature.jl` solution) where `x` is `a`, `b`, ...; e.g. 
```julia
f_a = x -> exp(-x^2)
I_a = my_method(f_a, ...) 
Icub_a, err_a = hquadrature(f_a, ...)
```

(a) $f_a(x) = e^{-x^2}, x \in [0, 1]$

(b) $f_b(x) = x \log(x), x \in [0, 1]$ 

(c) $f_c(x) = \sqrt{x} \log(x), x \in [0, 1]$ 

(d) $f_d(x) = \sqrt{x} \exp(- 0.1 x), x \in [1, \infty]$

I encourage you to use brute-force rather than get too clever about manually resolving the singularities. Let the computer do the work for you.

In [ ]:
using Cubature 

# YOUR CODE HERE